In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [ ]:
df = pd.read_csv('/Users/tumendemberelshalkhaan/Desktop/raw_data/Texas_Raw.csv')
ids = ["{:08d}".format(item) for item in df['SID Number'].tolist()]

In [34]:
tx = pd.DataFrame(columns =[])

In [35]:
for id in ids:
    url = 'https://inmate.tdcj.texas.gov//InmateSearch/viewDetail.action?sid=' + id
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    table = soup.find('table', class_ = 'tdcj_table')
    rows = table.find_all('tr')

    offdates = []
    offenses = []
    sendates = []
    counties = []
    cases = []
    sentences = []

    for row in rows[1:]:
        cols = row.find_all('td')
        offdates.append(cols[0].text)
        offenses.append(cols[1].text)
        sendates.append(cols[2].text)
        counties.append(cols[3].text)
        cases.append(cols[4].text)
        sentences.append(cols[4].text)

    crimes = [list(pair) for pair in zip(offenses, sentences, counties, sendates, offdates, cases)]

    rows = []
    for pair in crimes:

        row = {
            'ID': id,
            'Offense Description': pair[0],
            'Sentence Length': pair[1],
            'County': pair[2],
            'Sentence Date': pair[3],
            'Offense Date': pair[4],
            'Case Number': pair[5]
        }
        rows.append(row)
        
    tx = pd.concat([tx, pd.DataFrame(rows)])